# Example 1: Basic Message Structure
In this example, we demostrate various ways to create a messages object.  The messages object is a list.  EAch entry in the list is a dictionary that represents a message containing keys suchs as 'role' and 'content'.  

For example, a typical message structure would look like this.

   ~~~
       messages = [
            {"role": "system", "content": "You are an expert in contract analysis."},
            {"role": "user", "content": "Please summarize the contract."}
       ]
   ~~~

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from pprint import pprint

# Define the system message and user message
contract_extraction_prompt = "You are an expert in contract analysis. Extract key clauses from the provided text."
full_text = "This contract includes a confidentiality clause, a termination clause, and a liability clause."

# Here is an example of how to create a messages
messages =  [
     {"role": "system", "content": contract_extraction_prompt },
     {"role": "user", "content": "This contract includes a confidentiality clause, a termination clause, and a liability clause."}
]

# Example 1 
print("\nExample 1:\n")
print("\nNotice in this example because we are not using the SystemMessage or HumanMessage classes so we cannot use pretty_print()\n")
for m in messages:
    pprint(m) # We have to use the pprint to pretty print each message


# Example 1a - same outcome, but different approach
print("\nExample 1a - same outcome but different approach:")
print("Notice how we are using the SystemMessage and HumanMessage classes to create the messages, these classes do have the pretty_print method defined.\n")
messages = [SystemMessage(content=f"You are an expert in contract analysis. Extract key clauses from the provided text.", name="Model")]
messages.append(HumanMessage(content=full_text,name="Rick"))

for m in messages:
    m.pretty_print()


# Example 1b - sample outcome, but different approach
print("\nExample 1b same outcome but different approach:")
messages = [{"role": "system", "content": contract_extraction_prompt}]
messages.append({"role": "user", "content": full_text})

for m in messages:
     print(m)

print("\nNow, let's print the messges using various ways as it's important to understand this strcuture:\n")
print("## pprint(messages) ##\n")
pprint(f"{messages} \n")
print("\n## print(messages) ##\n")
print(messages)
print("\n## pprint(messages[0] ##\n")
pprint(messages[0])


Example 1:


Notice in this example because we are not using the SystemMessage or HumanMessage classes so we cannot use pretty_print()

{'content': 'You are an expert in contract analysis. Extract key clauses from '
            'the provided text.',
 'role': 'system'}
{'content': 'This contract includes a confidentiality clause, a termination '
            'clause, and a liability clause.',
 'role': 'user'}

Example 1a - same outcome but different approach:
Notice how we are using the SystemMessage and HumanMessage classes to create the messages, these classes do have the pretty_print method defined.

================================ System Message ================================
Name: Model

You are an expert in contract analysis. Extract key clauses from the provided text.
================================ Human Message =================================
Name: Rick

This contract includes a confidentiality clause, a termination clause, and a liability clause.

Example 1b same outcome

# Example 2: Using a System Message with a Custom Function
In this example, we define a function to modify the state before passing it to the model, allowing for dynamic system messages.

In [31]:
import os
from dotenv import load_dotenv
from pathlib import Path
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing import Type
from langchain.tools import tool
from pydantic import BaseModel

# Get root directory path
root_dir = Path().absolute().parent
env_path = root_dir / '.env'

# Load .env from root
load_dotenv(dotenv_path=env_path)
print(f"Loaded .env from {env_path}")

# Access variables
api_key = os.getenv('AZURE_OPENAI_API_KEY')
print(f"API Key: {api_key[:4] + '*' * 28 + api_key[-4:]}")

# Define the input schema for our tool
class ClauseAnalysisInput(BaseModel):
    clause_type: str
    clause_text: str

# Create a custom tool for analyzing contract clauses
@tool(args_schema=ClauseAnalysisInput)
def analyze_clause(clause_type: str, clause_text: str) -> str:
    """
    Analyzes a specific contract clause and provides insights.
    
    Args:
        clause_type: The type of clause (e.g., 'confidentiality', 'termination', 'liability')
        clause_text: The actual text of the clause to analyze
    
    Returns:
        str: Analysis of the clause including key points and potential risks
    """
    analysis_template = {
        'confidentiality': lambda text: f"Confidentiality Analysis:\n- Scope: {text[:50]}...\n- Duration: Searching for time specifications...\n- Obligations: Identifying key responsibilities...",
        'termination': lambda text: f"Termination Analysis:\n- Notice Period: Searching...\n- Grounds for Termination: Analyzing conditions...\n- Post-termination obligations: Reviewing...",
        'liability': lambda text: f"Liability Analysis:\n- Limitation Scope: {text[:50]}...\n- Cap Amount: Searching for monetary limits...\n- Exclusions: Identifying carve-outs..."
    }
    
    if clause_type.lower() in analysis_template:
        return analysis_template[clause_type.lower()](clause_text)
    return f"Unrecognized clause type: {clause_type}. Please specify one of: confidentiality, termination, or liability."

# Initialize the model
model = AzureChatOpenAI(model_name="gpt-4o", temperature=0.5) # Example model initialization

# Add our custom tool to the tools list
tools = [analyze_clause]

# Define a dynamic system message
def state_modifier(state):
    return [
        SystemMessage(content="""You are a legal assistant specialized in contract analysis. 
        Use the analyze_clause tool to provide detailed insights about specific contract clauses.
        Always analyze each clause type mentioned in the user's request."""),
        *state["messages"]
    ]

# Create the agent with the state modifier and tools
langgraph_agent_executor = create_react_agent(model, tools, state_modifier=state_modifier)

# Prepare user message
user_message = """Please analyze the following contract clauses:
1. Confidentiality: All information shared during the project shall remain confidential for 5 years.
2. Termination: Either party may terminate with 30 days notice.
3. Liability: Total liability shall not exceed the total contract value."""

# Create a HumanMessage object
human_message = HumanMessage(content=user_message)

# Invoke the agent with the user message
messages = langgraph_agent_executor.invoke({
    "messages": [human_message]
})

# Print the output from the agent
print("\nAgent Response:")
last_message = messages['messages'][-1]
if isinstance(last_message, AIMessage):
    print(last_message.content)
else:
    print("Unexpected message type:", type(last_message))


Loaded .env from c:\Users\rickcau\source\repos\vendor-contracts-gen-ai\.env
API Key: 8PVz****************************Isv1

Agent Response:
Here are the analyses for the specified contract clauses:

1. **Confidentiality Clause**:
   - **Scope**: The clause specifies that all information shared during the project must remain confidential. This implies a broad application to any data exchanged between the parties.
   - **Duration**: The confidentiality obligation is set for a period of 5 years, which is a reasonable timeframe for maintaining secrecy, although it may vary depending on industry standards and the nature of the information.
   - **Obligations**: The clause imposes a responsibility on both parties to protect confidential information, but it does not specify any particular measures or exceptions, which might be important to consider.

2. **Termination Clause**:
   - **Notice Period**: Either party can terminate the contract with a 30-day notice. This provides a clear and reason

# Example 3: Using Prompts with Message History
This example shows how to maintain a conversation history by appending messages to the list.

In [ ]:
# Initialize message history
messages = []

# Add system message
messages.append({"role": "system", "content": "You are a helpful assistant."})

# User asks a question
messages.append({"role": "user", "content": "What are the main points of the contract?"})

# AI responds (simulated)
ai_response = "The main points include confidentiality, termination, and liability."

# Append AI response to message history
messages.append({"role": "assistant", "content": ai_response})

# Continue the conversation
messages.append({"role": "user", "content": "Can you explain the confidentiality clause?"})


# Example 4: Using a System Message with a Prompt Template
In this example, we use a prompt template to control the agent's responses.

In [20]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Define a prompt template
prompt_template = "You are a contract expert. Analyze the following text: {text}"

# Create the agent
langgraph_agent_executor = create_react_agent(model, tools)

# Prepare the user message with the prompt template
contract_text = "This contract includes a confidentiality clause."
user_message = prompt_template.format(text=contract_text)

# Invoke the agent
messages = langgraph_agent_executor.invoke({"messages": [{"role": "user", "content": user_message}]})

# Print the output from the agent
print("\nAgent Response:")
last_message = messages['messages'][-1]
if isinstance(last_message, AIMessage):
    print(last_message.content)
else:
    print("Unexpected message type:", type(last_message))



Agent Response:
To provide a detailed analysis, I need the actual text of the confidentiality clause. Could you please provide the specific wording of the clause?


# Example 5: Streaming Messages
This example demostrates how to stream messages progressively, enhancing user experience.

In [33]:
# Initialize messages
messages = [{"role": "system", "content": "You are a legal assistant."}]

# User message
messages.append({"role": "user", "content": "Please summarize the contract."})

# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Stream the response
for chunk in langgraph_agent_executor.stream({"messages": messages}, config, stream_mode="updates"):
    print(chunk)  # Print the entire chunk to inspect its structure

for chunk in langgraph_agent_executor.stream({"messages": [HumanMessage(content="Please summarize the contract.")]}, config, stream_mode="updates"):
    print(chunk)  # Print the entire chunk to inspect its structure
    
    
for chunk in langgraph_agent_executor.stream({"messages": messages}, config, stream_mode="updates"):
    ai_messages = chunk['agent']['messages']  # Access the list of AI messages
    for ai_message in ai_messages:
        print(ai_message.content)  # Print the content of each AI message



{'agent': {'messages': [AIMessage(content="To summarize the contract, I'll need you to provide the text or main sections of the contract. Please share the relevant details or specific clauses you'd like summarized.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 160, 'total_tokens': 192, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 's

Here is a deeper example of streaming.  Notice how I can decide if I want to extract the AI Message response from the stream or the tool message.  What this means is that if I am using an API, I can avoid returning the tool messages to the client and only the AI Message.

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing import Type
from langchain.tools import tool
from pydantic import BaseModel

# Get root directory path
root_dir = Path().absolute().parent
env_path = root_dir / '.env'

# Load .env from root
load_dotenv(dotenv_path=env_path)
print(f"Loaded .env from {env_path}")

# Access variables
api_key = os.getenv('AZURE_OPENAI_API_KEY')
print(f"API Key: {api_key[:4] + '*' * 28 + api_key[-4:]}")

# Define the input schema for our tool
class ClauseAnalysisInput(BaseModel):
    clause_type: str
    clause_text: str

# Create a custom tool for analyzing contract clauses
@tool(args_schema=ClauseAnalysisInput)
def analyze_clause(clause_type: str, clause_text: str) -> str:
    """
    Analyzes a specific contract clause and provides insights.
    
    Args:
        clause_type: The type of clause (e.g., 'confidentiality', 'termination', 'liability')
        clause_text: The actual text of the clause to analyze
    
    Returns:
        str: Analysis of the clause including key points and potential risks
    """
    analysis_template = {
        'confidentiality': lambda text: f"Confidentiality Analysis:\n- Scope: {text[:50]}...\n- Duration: Searching for time specifications...\n- Obligations: Identifying key responsibilities...",
        'termination': lambda text: f"Termination Analysis:\n- Notice Period: Searching...\n- Grounds for Termination: Analyzing conditions...\n- Post-termination obligations: Reviewing...",
        'liability': lambda text: f"Liability Analysis:\n- Limitation Scope: {text[:50]}...\n- Cap Amount: Searching for monetary limits...\n- Exclusions: Identifying carve-outs..."
    }
    
    if clause_type.lower() in analysis_template:
        return analysis_template[clause_type.lower()](clause_text)
    return f"Unrecognized clause type: {clause_type}. Please specify one of: confidentiality, termination, or liability."

# Initialize the model
model = AzureChatOpenAI(model_name="gpt-4o", temperature=0.5) # Example model initialization

# Add our custom tool to the tools list
tools = [analyze_clause]

# Define a dynamic system message
def state_modifier(state):
    return [
        SystemMessage(content="""You are a legal assistant specialized in contract analysis. 
        Use the analyze_clause tool to provide detailed insights about specific contract clauses.
        Always analyze each clause type mentioned in the user's request."""),
        *state["messages"]
    ]

# Create the agent with the state modifier and tools
langgraph_agent_executor = create_react_agent(model, tools, state_modifier=state_modifier)

# Prepare user message
user_message = (
    "Please analyze the following contract clauses in detail: "
    "1. Confidentiality Clause: All information shared during the project shall remain confidential for 5 years. "
    "2. Termination Clause: Either party may terminate with 30 days notice. "
    "3. Liability Clause: Total liability shall not exceed the total contract value. "
    "For each clause, provide a detailed analysis, including implications, potential risks, and any relevant legal considerations."
)

# Create a HumanMessage object
human_message = HumanMessage(content=user_message)

# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Stream the response - Sample example as earlier but using streaming

# Stream the response
print("Streaming response for the detailed analysis:")
for chunk in langgraph_agent_executor.stream({"messages": [human_message]}, config, stream_mode="updates"):
    # Check if the chunk contains agent messages
    if 'agent' in chunk:
        ai_messages = chunk['agent']['messages']
        print("AI Messages:")
        for ai_message in ai_messages:
            print(ai_message.content)  # Print the content of each AI message
    # Check if the chunk contains tool messages
    if 'tools' in chunk:
        tool_messages = chunk['tools']['messages']
        print("Tool Messages:")
        for tool_message in tool_messages:
            print(tool_message.content)  # Print the content of each ToolMessage



Loaded .env from c:\Users\rickcau\source\repos\vendor-contracts-gen-ai\.env
API Key: 8PVz****************************Isv1
Streaming response for the detailed analysis:
AI Messages:

Tool Messages:
Confidentiality Analysis:
- Scope: All information shared during the project shall re...
- Duration: Searching for time specifications...
- Obligations: Identifying key responsibilities...
Termination Analysis:
- Notice Period: Searching...
- Grounds for Termination: Analyzing conditions...
- Post-termination obligations: Reviewing...
Liability Analysis:
- Limitation Scope: Total liability shall not exceed the total contrac...
- Cap Amount: Searching for monetary limits...
- Exclusions: Identifying carve-outs...
AI Messages:
Here's a detailed analysis of each clause:

### Confidentiality Clause
- **Scope**: The clause covers all information shared during the project, indicating a broad scope. This means any data, documents, or verbal communications could be considered confidential.
- **Durati